# Lab 30: Ransomware Fundamentals

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab30_ransomware_fundamentals.ipynb)

Understand ransomware families, attack lifecycles, and detection indicators.

## Learning Objectives
- Understand ransomware evolution and major families
- Learn the ransomware attack lifecycle
- Identify key indicators of compromise (IOCs)
- Use AI to analyze and attribute ransomware

## Educational Purpose Only

This lab teaches **defensive** techniques. All code is for learning.

**Next:** Lab 31 (Ransomware Detection)

In [ ]:
#@title Install dependencies (Colab only)
%pip install -q anthropic openai google-generativeai python-dotenv pandas matplotlib

In [ ]:
import os
import json
import pandas as pd

try:
    from google.colab import userdata
    for key in ["ANTHROPIC_API_KEY", "OPENAI_API_KEY", "GOOGLE_API_KEY"]:
        try:
            os.environ[key] = userdata.get(key)
        except:
            pass
except:
    pass

def setup_llm():
    if os.environ.get("ANTHROPIC_API_KEY"):
        return "anthropic", "claude-sonnet-4.5"
    elif os.environ.get("OPENAI_API_KEY"):
        return "openai", "gpt-5"
    elif os.environ.get("GOOGLE_API_KEY"):
        return "google", "gemini-3-flash"
    raise ValueError("No API key found.")

def query_llm(prompt: str, system: str = "You are a ransomware analyst.") -> str:
    provider, model = setup_llm()
    if provider == "anthropic":
        from anthropic import Anthropic
        client = Anthropic()
        response = client.messages.create(
            model=model, max_tokens=2048, system=system,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.content[0].text
    elif provider == "openai":
        from openai import OpenAI
        client = OpenAI()
        response = client.chat.completions.create(
            model=model, max_tokens=2048,
            messages=[{"role": "system", "content": system}, {"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    elif provider == "google":
        import google.generativeai as genai
        genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
        model_instance = genai.GenerativeModel(model)
        response = model_instance.generate_content(f"{system}\n\n{prompt}")
        return response.text

provider, model = setup_llm()
print(f"✅ Using {provider} ({model})")

## Ransomware Evolution

| Era | Period | Characteristics | Examples |
|-----|--------|-----------------|----------|
| **Early** | 2013-2016 | Simple encryption, Bitcoin payments | CryptoLocker, TeslaCrypt |
| **RaaS** | 2017-2020 | Ransomware-as-a-Service, affiliates | GandCrab, REvil, Ryuk |
| **Double Extortion** | 2020+ | Data theft + encryption | Maze, DarkSide, LockBit |
| **Triple Extortion** | 2021+ | + DDoS, customer harassment | ALPHV, Clop, Royal |

In [ ]:
# Major ransomware families database
RANSOMWARE_FAMILIES = {
    "LockBit": {
        "first_seen": "2019",
        "model": "RaaS",
        "encryption": "AES + RSA",
        "extension": ".lockbit",
        "ransom_note": "Restore-My-Files.txt",
        "notable_ttps": ["Fast encryption", "Self-spreading", "AD exploitation"],
        "active": True
    },
    "ALPHV (BlackCat)": {
        "first_seen": "2021",
        "model": "RaaS",
        "encryption": "AES + RSA (Rust-based)",
        "extension": "Random 7 chars",
        "ransom_note": "RECOVER-[id]-FILES.txt",
        "notable_ttps": ["Cross-platform", "Data leak site", "Triple extortion"],
        "active": False  # Disrupted 2024
    },
    "Conti": {
        "first_seen": "2020",
        "model": "RaaS",
        "encryption": "AES-256 + RSA-4096",
        "extension": ".CONTI",
        "ransom_note": "readme.txt",
        "notable_ttps": ["Manual hacking", "Cobalt Strike", "Domain takeover"],
        "active": False  # Rebranded 2022
    },
    "Akira": {
        "first_seen": "2023",
        "model": "RaaS",
        "encryption": "ChaCha20 + RSA",
        "extension": ".akira",
        "ransom_note": "akira_readme.txt",
        "notable_ttps": ["VPN exploitation", "Linux targeting", "Retro website"],
        "active": True
    }
}

# Display as DataFrame
df = pd.DataFrame(RANSOMWARE_FAMILIES).T
print("📊 Major Ransomware Families")
df

## Ransomware Attack Lifecycle

```
┌─────────────────────────────────────────────────────────────────────┐
│                    RANSOMWARE ATTACK LIFECYCLE                       │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  1. INITIAL ACCESS          2. EXECUTION           3. PERSISTENCE   │
│  ┌─────────────────┐        ┌──────────────┐      ┌──────────────┐  │
│  │ • Phishing      │   →    │ • Loader     │  →   │ • Registry   │  │
│  │ • RDP brute     │        │ • PowerShell │      │ • Scheduled  │  │
│  │ • VPN exploit   │        │ • WMI        │      │   tasks      │  │
│  └─────────────────┘        └──────────────┘      └──────────────┘  │
│                                    │                                 │
│                                    ▼                                 │
│  6. IMPACT                 5. EXFILTRATION      4. DISCOVERY        │
│  ┌─────────────────┐      ┌──────────────┐     ┌──────────────┐     │
│  │ • Encryption    │  ←   │ • Cloud sync │  ←  │ • AD recon   │     │
│  │ • Ransom note   │      │ • FTP/HTTP   │     │ • File enum  │     │
│  │ • Backup delete │      │ • RClone     │     │ • Net shares │     │
│  └─────────────────┘      └──────────────┘     └──────────────┘     │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Simulated ransomware IOCs
SAMPLE_IOCS = {
    "file_extensions": [".lockbit", ".encrypted", ".akira", ".PLAY"],
    "ransom_notes": [
        "HOW_TO_DECRYPT.txt",
        "Restore-My-Files.txt",
        "!!! READ ME !!!.txt",
        "akira_readme.txt"
    ],
    "suspicious_processes": [
        "vssadmin.exe delete shadows",
        "wmic.exe shadowcopy delete",
        "bcdedit.exe /set {default} recoveryenabled No",
        "cipher.exe /w:"
    ],
    "registry_keys": [
        "HKCU\\Software\\Microsoft\\Windows\\CurrentVersion\\Run",
        "HKLM\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\RunOnce"
    ]
}

print("🔍 Common Ransomware IOCs")
for category, iocs in SAMPLE_IOCS.items():
    print(f"\n{category.upper()}:")
    for ioc in iocs[:3]:
        print(f"  • {ioc}")

## Using AI for Ransomware Analysis

In [ ]:
# Simulated ransom note for analysis
SAMPLE_RANSOM_NOTE = """
Your network has been breached and all data has been encrypted.

All your files, documents, photos, databases, and other important files 
have been encrypted with military grade encryption algorithm.

The only method of recovering files is to purchase decrypt tool and unique key for you.

DO NOT MODIFY ENCRYPTED FILES
DO NOT USE THIRD PARTY SOFTWARE TO RESTORE YOUR DATA

Contact us: decryption@onionmail.org
Your ID: aB3x7-Km9Pq-2Rf4W

Price: 2 BTC
Wallet: bc1qxy2kgdygjrsqtzq2n0yrf2493p83kkfjhx0wlh

If you do not contact us within 72 hours, we will publish your data.
"""

print("📝 Sample Ransom Note:")
print(SAMPLE_RANSOM_NOTE)

In [ ]:
ANALYSIS_PROMPT = """
Analyze this ransom note and extract the following:

1. **Indicators of Compromise (IOCs)**
   - Email addresses
   - Bitcoin wallets
   - Victim IDs
   - Onion URLs

2. **Ransomware Family Attribution**
   - Which family does this likely belong to?
   - Confidence level and reasoning

3. **Threat Assessment**
   - Is double extortion mentioned?
   - Time pressure tactics?
   - Ransom amount assessment

4. **Recommendations**
   - Immediate actions for the victim
   - Evidence to preserve

Ransom Note:
```
{note}
```
"""

print("🔬 AI Ransomware Analysis")
print("=" * 50)

analysis = query_llm(ANALYSIS_PROMPT.format(note=SAMPLE_RANSOM_NOTE))
print(analysis)

## Recovery Decision Framework

In [ ]:
def assess_recovery_options(backup_status: str, data_criticality: str, 
                           ransom_amount: float, decryptor_available: bool) -> dict:
    """Assess recovery options for ransomware incident."""
    
    options = {
        "restore_from_backup": {
            "viable": backup_status in ["clean", "partial"],
            "time_estimate": "Hours to days",
            "cost": "IT labor",
            "data_loss": "Since last backup" if backup_status == "clean" else "Partial"
        },
        "use_decryptor": {
            "viable": decryptor_available,
            "time_estimate": "Hours",
            "cost": "Free",
            "data_loss": "None",
            "note": "Check nomoreransom.org"
        },
        "pay_ransom": {
            "viable": True,  # Always possible but not recommended
            "time_estimate": "Days (if they cooperate)",
            "cost": f"${ransom_amount:,.0f} + negotiations",
            "data_loss": "Unknown - no guarantee",
            "risks": ["May not get decryptor", "Funds criminal enterprise", "May be targeted again"]
        },
        "rebuild": {
            "viable": True,
            "time_estimate": "Weeks to months",
            "cost": "Significant IT labor",
            "data_loss": "All encrypted data"
        }
    }
    
    # Recommendation logic
    if decryptor_available:
        recommendation = "use_decryptor"
    elif backup_status == "clean":
        recommendation = "restore_from_backup"
    elif backup_status == "partial" and data_criticality != "critical":
        recommendation = "restore_from_backup"
    else:
        recommendation = "rebuild"  # Never recommend paying
    
    return {
        "options": options,
        "recommendation": recommendation,
        "note": "FBI recommends against paying ransoms"
    }

# Example scenario
result = assess_recovery_options(
    backup_status="partial",
    data_criticality="high",
    ransom_amount=100000,
    decryptor_available=False
)

print("🔄 Recovery Assessment")
print(f"Recommended: {result['recommendation'].upper()}")
print(f"Note: {result['note']}")

## Exercises

### Exercise 1: IOC Extraction
Build a function that extracts all IOCs from a ransom note (emails, BTC addresses, etc.)

### Exercise 2: Family Classifier
Create a simple classifier that identifies ransomware family from file extension + ransom note.

### Exercise 3: Timeline Builder
Given a list of event logs, use AI to build a ransomware incident timeline.

## Next Steps

- **Lab 31**: Build a ransomware detection system using ML
- **Lab 12**: Run purple team ransomware simulations
- **Lab 13**: Memory forensics for ransomware analysis